# Introduction to Deep Learning Laboratories

<a href="https://creativecommons.org/licenses/by-nc/4.0/">
  <img src="https://licensebuttons.net/l/by-nc/4.0/88x31.png" alt="CC BY-NC 4.0" style="vertical-align:middle;">
</a>
<span style="vertical-align:middle; margin-left:8px;">
  <strong>CC BY‑NC 4.0</strong> — non‑commercial use with attribution.  
  Full terms at the link.
</span>

<br>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/antoninofurnari/deeplearning/blob/main/notebooks/03-03-26-intro-laboratories.ipynb
)

**Course:** Deep Learning: Advanced Models and Methods

**Objective:**
Welcome to the course! Today, we aren't just checking our environment; we are establishing the workflow for the entire semester. Deep Learning is as much about engineering discipline as it is about mathematics.

**In this lab, we will:**
1.  **Understand the Notebook Philosophy:** Learn the visual language of these labs.
2.  **Verify the Forge:** Ensure PyTorch and CUDA are communicating on Google Colab.
3.  **Connect the Logger:** Initialize Weights & Biases (W&B) for experiment tracking.
4.  **The Cluster Guide:** A reference guide for moving from Colab to the University Cluster.
5.  **First Strike:** Train your first PyTorch Lightning model (MLP on MNIST) and log it.

## The Rules of Engagement

Throughout this course, we will use specific visual cues to guide your learning. Please familiarize yourself with this legend:

<div class="alert alert-block alert-info">
<b>☑️ TODO</b> This box provides you instruction on a TODO that should be completed in the code cells below.
</div>

<div class="alert alert-block alert-danger">
<b>🛑 Checkpoint</b> The following cells provide you instruction on what you should observe if you run a given cell after having implemented your TODOs. This serve as a self-check to assess potential bugs or errors early.
</div>

<div class="alert alert-block alert-warning">
<b>📝 Reflection</b> You are required to write a short reflection on your results.
</div>

<div class="alert alert-block alert-success">
<b>🏁 Milestone</b> This signal that you completed a significant part of the notebook and reached a milestone.
</div>

Throughout the course, we will use Weight and Biases for logging: http://wandb.ai/

If you use colab, you will need to install it as follows:

In [4]:
# CELL | PURPOSE: Install necessary libraries
# We need PyTorch Lightning for structure and WandB for logging.
# The 'quiet' flag reduces the log noise.
!pip install lightning wandb -q

## **Part 1: The Local Forge (Colab Setup)**

Google Colab provides us with a free (but time-limited) GPU. Let's inspect what hardware we were assigned and ensure PyTorch can see it.

We also use PyTorch and Lightning during the course. Let's check their versions.

In [5]:
# CELL | PURPOSE: Verify Environment
import torch
import lightning as L
import wandb
import os

print(f"PyTorch Version: {torch.__version__}")
print(f"Lightning Version: {L.__version__}")

# Check GPU availability
if torch.cuda.is_available():
    print(f"✅ Success! CUDA is available.")
    print(f"   GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"   Capability: {torch.cuda.get_device_capability(0)}")
else:
    print("⚠️ WARNING: You are running on CPU. Go to Runtime -> Change runtime type -> Hardware accelerator -> T4 GPU.")

# Set a global seed for reproducibility (Crucial for scientific rigor)
L.seed_everything(42)

Seed set to 42


PyTorch Version: 2.6.0+cu124
Lightning Version: 2.6.0
✅ Success! CUDA is available.
   GPU Name: Tesla V100-PCIE-16GB
   Capability: (7, 0)


42

Let's now login to wandb:

In [6]:
# CELL | PURPOSE: Login to Weights & Biases
# You will be prompted to paste your API key from https://wandb.ai/authorize
wandb.login()

True

## **Part 2: Lightning Strike (Introduction to PyTorch Lightning)**

We use **PyTorch Lightning** to organize our code. It separates the *science* (the model architecture) from the *engineering* (the training loop).

**The Task:** Train a simple Multi-Layer Perceptron (MLP) to classify handwritten digits (MNIST).

<div class="alert alert-info">
  <b>☑️ TODO:</b> In the cell below, fill in the missing code to define the optimizer.
</div>

In [ ]:
# CELL | PURPOSE: Define the LightningModule
from torch import nn
from torch.nn import functional as F

class MNISTClassifier(L.LightningModule):
    def __init__(self, hidden_dim=128, learning_rate=1e-3):
        super().__init__()
        self.save_hyperparameters() # Logs params to W&B automatically

        # 1. Define Architecture (Simple MLP)
        self.net = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28 * 28, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 10) # 10 classes for digits 0-9
        )
        self.learning_rate = learning_rate

    def forward(self, x):
        return self.net(x)

    def training_step(self, batch, batch_idx):
        # The "Magic" Loop: No need to manual zero_grad or backward()
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)

        # Log to W&B
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == y).float().mean()

        # Log validation metrics
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == y).float().mean()
        self.log("test_acc", acc)

    def configure_optimizers(self):
        # TODO: Define the Adam optimizer using self.learning_rate
        # Hint: Use torch.optim.Adam
        # < YOUR CODE HERE >
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        # </ YOUR CODE HERE >
        return optimizer

print("✅ Model Class Defined Successfully.")

✅ Model Class Defined Successfully.


### **Data Preparation**
Lightning handles the training loop, but we still need standard PyTorch DataLoaders. We will use a batch size of 256 to maximize GPU throughput.

In [8]:
# CELL | PURPOSE: Prepare DataLoaders
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST
from torchvision import transforms

# Configuration
BATCH_SIZE = 256

# Download MNIST
print("📦 Downloading Data...")
transform = transforms.ToTensor()
dataset = MNIST(root="./data", train=True, download=True, transform=transform)
mnist_test = MNIST(root="./data", train=False, download=True, transform=transform)

# Split Train/Val
mnist_train, mnist_val = random_split(dataset, [55000, 5000])

# Create Loaders
train_loader = DataLoader(mnist_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(mnist_val, batch_size=BATCH_SIZE, num_workers=2)
test_loader = DataLoader(mnist_test, batch_size=BATCH_SIZE, num_workers=2)

print("✅ Data Loaded. Train size:", len(mnist_train), "Val size:", len(mnist_val))

📦 Downloading Data...
✅ Data Loaded. Train size: 55000 Val size: 5000


### **Training with W&B**
Now we invoke the **Trainer**. This object handles GPU placement, checkpointing, and logging automatically.


<div class="alert alert-block alert-danger">
<b>🛑 Checkpoint</b> Run the cell below. You should see a progress bar and a link to your W&B dashboard. Click the link to see your loss curves in real-time!
</div>

In [9]:
# CELL | PURPOSE: Run Training
from lightning.pytorch.loggers import WandbLogger

# 1. Init Logger
wandb_logger = WandbLogger(project="DL_Lab_0_Intro", name="MNIST_MLP_Run")

# 2. Init Model
model = MNISTClassifier(hidden_dim=128, learning_rate=0.001)

# 3. Init Trainer
# accelerator="auto" automatically finds the T4 GPU.
trainer = L.Trainer(
    max_epochs=5,
    accelerator="auto",
    devices=1,
    logger=wandb_logger,
    enable_progress_bar=True
)

# 4. Train
trainer.fit(model, train_loader, val_loader)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
wandb: WARNING Unable to render HTML, can't import display from ipython.core
wandb: WARNING Unable to render HTML, can't import display from ipython.core
wandb: WARNING Unable to render HTML, can't import display from ipython.core
wandb: WARNING Unable to render HTML, can't import display from ipython.core
wandb: WARNING Unable to render HTML, can't import display from ipython.core
wandb: WARNING Unable to render HTML, can't import display from ipython.core
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


┏━━━┳━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name ┃ Type       ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ net  │ Sequential │  101 K │ train │     0 │
└───┴──────┴────────────┴────────┴───────┴───────┘

Trainable params: 101 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 101 K                                                                                                
Total estimated model params size (MB): 0                                                                          
Modules in train mode: 5                                                                                           
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

`Trainer.fit` stopped: `max_epochs=5` reached.


<div class="alert alert-block alert-warning">
<b>📝 Reflection</b> 

<i>Look at the training curves on Weight and Biases and briefly comment in the following.</i>

Your Text
</div>

<div class="alert alert-block alert-success">
<b>🏁 Milestone</b> You successfully completed training!
</div>

## **Part 3: The Heavy Artillery (The Cluster)**

While Colab is excellent for debugging and small-scale experiments, your major projects may require the University Cluster (`gcluster`).

> NOTE: You do not need to run this section today. This is your reference guide for later in the semester.

You can find the relevant information in the documentation: https://gcluster.dmi.unict.it/docs/